# NLP project

In this project I will use NLP on the IMDB dataset. The first step is to read this dataset and prepare it for the NLP model.

In [1]:
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.layers import Input, LSTM, Dense, Embedding
from keras.models import Model
from keras.utils import pad_sequences

# Read in the dataset
data = pd.read_csv("IMDB Dataset.csv")



In [2]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [3]:
# Convert text to lowercase
data['review'] = data['review'].apply(lambda x: x.lower())

# Tokenize the text
review_tokenizer = Tokenizer()
review_tokenizer.fit_on_texts(data['review'])
sentiment_tokenizer = Tokenizer()
sentiment_tokenizer.fit_on_texts(data['sentiment'])

# Convert text to sequences of integers
review_sequences = review_tokenizer.texts_to_sequences(data['review'])
sentiment_sequences = sentiment_tokenizer.texts_to_sequences(data['sentiment'])

# Pad sequences to a fixed length
max_sequence_length = 100
review_data = pad_sequences(review_sequences, maxlen=max_sequence_length, padding='post')
sentiment_data = pad_sequences(sentiment_sequences, maxlen=1, padding='post')

In this code, we first read in the dataset. We then convert the text to lowercase and tokenize the text using Keras' Tokenizer class. We also pad the sequences to a fixed length of 100.

Next, we split the data into training and validation sets.

In [4]:
review_data

array([[   34,  1637,     9, ...,   125,  4103,   486],
       [ 9719,    31,     1, ...,  1977,    69,   221],
       [ 3059,    12,  2971, ...,    63,    16,   350],
       ...,
       [   26,     3,  1156, ..., 22840,     2,  6050],
       [    5,    68,   135, ...,    67,   739,    42],
       [  699,   479,    11, ...,   794,    11,    17]])

In [5]:
# start en stop token erbij

sentiment_data

array([[1],
       [1],
       [1],
       ...,
       [2],
       [2],
       [2]])

In [6]:
# Split the data into training and validation sets
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(review_data, sentiment_data, test_size=0.2, random_state=42)


We can now define our encoder and decoder models.

In [7]:
# Define input sequence
encoder_inputs = Input(shape=(None,))
# Define output sequence
decoder_inputs = Input(shape=(None,))

# Define encoder embedding layer
encoder_embedding = Embedding(len(review_tokenizer.word_index) + 1, 256)
encoder_embedding_output = encoder_embedding(encoder_inputs)

# Define encoder LSTM layer
encoder_lstm = LSTM(256, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding_output)
encoder_states = [state_h, state_c]

# Define decoder embedding layer
decoder_embedding = Embedding(len(sentiment_tokenizer.word_index) + 1, 256)
decoder_embedding_output = decoder_embedding(decoder_inputs)

# Define decoder LSTM layer
decoder_lstm = LSTM(256, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding_output, initial_state=encoder_states)

# Define output layer
decoder_dense = Dense(len(sentiment_tokenizer.word_index) + 1, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In this code, we define the input and output sequences, as well as the embedding and LSTM layers for the encoder and decoder. We also define the output layer and the entire model.

We can now compile and train the model.

In [8]:
from keras.utils import to_categorical


# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train model
# Define batch size and number of epochs
batch_size = 64
epochs = 10

# Define generator for training data
def generate_batch(X=X_train, y=y_train, batch_size=batch_size):
    while True:
        for i in range(0, len(X), batch_size):
            encoder_input_data = X[i:i + batch_size]
            decoder_input_data = y[i:i + batch_size, :-1]
            decoder_output_data = y[i:i + batch_size, 1:]
            encoder_input_data = np.array(encoder_input_data)
            decoder_input_data = np.array(decoder_input_data)
            decoder_output_data = np.array(decoder_output_data)
            decoder_output_data = to_categorical(decoder_output_data, num_classes=len(sentiment_tokenizer.word_index) + 1)
            yield ([encoder_input_data, decoder_input_data], decoder_output_data)

# Define generator for validation data
def generate_validation(X=X_val, y=y_val):
    encoder_input_data = np.array(X)
    decoder_input_data = np.array(y[:, :-1])
    decoder_output_data = np.array(y[:, 1:])
    decoder_output_data = to_categorical(decoder_output_data, num_classes=len(sentiment_tokenizer.word_index) + 1)
    return ([encoder_input_data, decoder_input_data], decoder_output_data)

# Train model
model.fit_generator(generator=generate_batch(),
                    steps_per_epoch=len(X_train)//batch_size,
                    epochs=epochs,
                    validation_data=generate_validation(),
                    validation_steps=len(X_val)//batch_size)

Epoch 1/10


C:\Users\heath\AppData\Local\Temp\ipykernel_84\2935895805.py:34: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator=generate_batch(),


InvalidArgumentError: Graph execution error:

Tried to stack elements of an empty list with non-fully-defined element_shape: [?,256]
	 [[{{node TensorArrayV2Stack/TensorListStack}}]]
	 [[model/lstm_1/PartitionedCall]] [Op:__inference_train_function_6228]

In this code, we define the batch size and number of epochs. We also define the generators for the training and validation data. The generate_batch function generates batches of data for the training data. The generate_validation function generates data for the validation data. We then train the model using the fit_generator function.

Finally, we can make predictions using the trained model.

In [ ]:
# Define encoder model
encoder_model = Model(encoder_inputs, encoder_states)

# Define decoder inputs
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_inputs = Input(shape=(None,))
decoder_embedding_output = decoder_embedding(decoder_inputs)
decoder_outputs, state_h, state_c = decoder_lstm(decoder_embedding_output, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

def decode_sequence(input_seq):
    # Encode the input sequence to get the encoder states
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1
    target_seq = np.zeros((1, 1))

    # Populate the first character of target sequence with the start character
    target_seq[0, 0] = sentiment_tokenizer.word_index['<start>']

    # Generate output sequence
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = sentiment_tokenizer.index_word[sampled_token_index]

        # Exit condition: either hit max length or find stop character
        if (sampled_token == '<end>' or len(decoded_sentence) > max_sequence_length):
            stop_condition = True
        else:
            decoded_sentence += ' ' + sampled_token

            # Update the target sequence
            target_seq = np.zeros((1, 1))
            target_seq[0, 0] = sampled_token_index

            # Update states
            states_value = [h, c]

    return decoded_sentence



In this code, we define a function sequence_to_text that converts a sequence to text. We then generate some translations for the validation data by selecting a random input sequence, using the decode_sequence function to generate the predicted output sequence, and then converting the sequences to text. We print the input, target, and predicted sequences for each example.

In [ ]:
def sequence_to_text(sequence, tokenizer):
    """
    Converts a sequence of integers to its corresponding text sequence.
    
    Args:
    - sequence (np.array): A sequence of integers.
    - tokenizer (keras.preprocessing.text.Tokenizer): A tokenizer fitted on the text data.
    
    Returns:
    - A string representing the text sequence.
    """
    text = tokenizer.sequences_to_texts([sequence])[0]
    return text


In [ ]:
# Define a new review
new_review = "This movie is fantastic. I loved every minute of it!"

# Convert the review to a sequence of integers
new_review_seq = review_tokenizer.texts_to_sequences([new_review])[0]

# Pad the sequence to the same length as the training data
new_review_seq = pad_sequences([new_review_seq], maxlen=max_sequence_length, padding='post')

# Use the model to predict the sentiment of the review
pred_sentiment_seq = model.predict([new_review_seq, np.zeros((len(new_review_seq), 1))])

# Convert the predicted sentiment sequence to text
pred_sentiment_text = sequence_to_text(np.argmax(pred_sentiment_seq, axis=2)[0], sentiment_tokenizer)

print("Review:", new_review)
print("Predicted sentiment:", pred_sentiment_text)


1/1 [==============================] - 0s 405ms/step
Review: This movie is fantastic. I loved every minute of it!
Predicted sentiment: 
